In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm

In [27]:
import numpy as np
from scipy.stats import norm
import datetime as dt
import matplotlib.pyplot as plt

def C_v(S,K,T,r,q,sigma):
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #q: dividend rate
    #sigma: volatility of underlying asset
    
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    result = (S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2))        
    return result

def P_v(S,K,T,r,q,sigma):
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #q: dividend rate
    #sigma: volatility of underlying asset
    
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    result = (K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1))        
    return result

def C_d(S,K,T,r,q,sigma):
    d2 = (np.log(S / K) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return np.exp(-r*T)*norm.cdf(d2)

def P_d(S,K,T,r,q,sigma):
    d2 = (np.log(S / K) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return np.exp(-r*T)*norm.cdf(-d2)

#https://people.maths.ox.ac.uk/howison/barriers.pdf

def C_do(S,K,T,r,q,sigma,BL):
    if S<BL:
        return 0.0
        
    s2 = sigma**2
    k = 2*(r-q)/s2
    a = 0.5*(1-k)
    b = -0.25*(k-1)**2-k
    if BL < K:
        cbs = C_v(S,K,T,r,q,sigma)        
        return  cbs - (S/BL)**(2*a)*C_v(BL**2/S,K,T,r,q,sigma)
    else:
    
        return (C_v(S,BL,T,r,q,sigma) + (BL-K)*C_d(S,BL,T,r,q,sigma) 
               -(S/BL)**(2*a)*(C_v(BL**2/S,BL,T,r,q,sigma) + (BL-K)*C_d(BL**2/S,BL,T,r,q,sigma))
               )

 


def C_di(S,K,T,r,q,sigma,BL):
    s2 = sigma**2
    k = 2*(r-q)/s2
    a = 0.5*(1-k)
    b = -0.25*(k-1)**2-k
    return (S/BL)**(2*a)*C_v(BL**2/S,K,T,r,q,sigma)


def P_do(S,K,T,r,q,sigma,BL):
    if S<BL:
        return 0.0

    s2 = sigma**2
    k = 2*(r-q)/s2
    a = 0.5*(1-k)
    b = -0.25*(k-1)**2-k

    return (P_v(S,K,T,r,q,sigma) - P_v(S,BL,T,r,q,sigma)  - (K-BL)*P_d(S,BL,T,r,q,sigma) 
            - (S/BL)**2 * (P_v(BL**2/S,K,T,r,q,sigma) - P_v(BL**2/S,BL,T,r,q,sigma) - (K-BL)*P_d(BL**2/S,BL,T,r,q,sigma) )
            )

def C_uo(S,K,T,r,q,sigma,BL):
    if S>BL:
        return 0.0

    s2 = sigma**2
    k = 2*(r-q)/s2
    a = 0.5*(1-k)
    b = -0.25*(k-1)**2-k

    npv = (C_v(S,K,T,r,q,sigma) - C_v(S,BL,T,r,q,sigma)  - (BL-K)*C_d(S,BL,T,r,q,sigma) 
            - (S/BL)**2 * (C_v(BL**2/S,K,T,r,q,sigma) - C_v(BL**2/S,BL,T,r,q,sigma) - (BL-K)*C_d(BL**2/S,BL,T,r,q,sigma) )
            )
    return npv

def P_uo(S,K,T,r,q,sigma,BL):
    if S>BL:
        return 0.0

    s2 = sigma**2
    k = 2*(r-q)/s2
    a = 0.5*(1-k)
    b = -0.25*(k-1)**2-k
    if BL>K:  
        npv = P_v(S,K,T,r,q,sigma)  - (S/BL)**(2*a)*P_v(BL**2/S,K,T,r,q,sigma)
  
    else:
        npv = (P_v(S,BL,T,r,q,sigma) + (K-BL)*P_d(S,BL,T,r,q,sigma) 
                -(S/BL)**(2*a)*(P_v(BL**2/S,BL,T,r,q,sigma) + (K-BL)*P_d(BL**2/S,BL,T,r,q,sigma))
                )
    return npv



In [63]:
np.random.seed(42)

class Asset:
    def __init__(self, K,sigma, payoff,notional, T):
        self.sigma = sigma
        self.K = K
        self.payoff=payoff
        self.notional = notional
        self.T = T

    def loss(self,S,r, T):
        return self.payoff(S,self.K,T,r,0.0,self.sigma)-self.payoff(self.K,self.K,T,r,0.0,self.sigma)


portfolio = [Asset(K=100,sigma=0.15, payoff=C_v, notional=1000, T = 0.1),
             Asset(K=100,sigma=0.35, payoff=C_v, notional=1000, T = 0.10),
             Asset(K=100,sigma=0.45,payoff=C_v, notional=1000, T = 0.10 )]

def compute_portfolio_loss(xportfolio,xS,xr,xT):
    return np.sum([p.notional*p.loss(s,r,t) for (p,s,r,t) in  zip(xportfolio,xS,xr,xT)])

def get_simulation_cube(nAssets, nSims, mu,Cov):
    rn = norm.rvs(size=(nAssets,nSims)) + mu.transpose()
    C = np.linalg.cholesky(Cov)

    lr = []
    g = 0.5*mu*mu.transpose()
    #print(f"f={g}")
    g = g[0,0]
    for i in range(rn.shape[1]):
        n=rn[:,i]
        #print(f"n={n}")
        h = np.matmul(mu,n)
        #print(f"h={h}")
        ff = g-h[0,0]
        print(f"ff={ff}")
        lr.append(np.exp(ff))
    
    lr=np.array(lr)
    rnorm = np.matmul(C,rn)
    return lr,rnorm

def get_losses(cube):
    loss=[]
    for i in range(cube.shape[1]):

def get_approx_mu(sigma, rho,x):
    l = np.diag(sigma)
    Cov = np.matmul(np.matmul(l,rho),l)
    S=np.array([p.K for p in portfolio])
    T=np.array([p.T for p in portfolio])
    r = np.array([0.01,0.01,0.01])
    
    
    a0 = compute_portfolio_loss(xportfolio=portfolio,xS = S, xr=r,xT=T)
    a1 = compute_portfolio_loss(xportfolio=portfolio,xS = S, xr=r,xT = T+0.01)
    a = -(a1 - a0)/0.01*0.0
    
    L = np.linalg.linalg.cholesky(Cov)
    Cp = np.matmul(L,L.transpose())
    print(Cp)
    print(Cov)
    d = []
    for i in range(len(portfolio)):        
        Sp = S.copy()
        Sp[i] *= 1.01
        xp = compute_portfolio_loss(xportfolio=portfolio,xS = Sp, xr=r, xT=T)
        Sm = S.copy()
        Sm[i] *= 0.99
        xm = compute_portfolio_loss(xportfolio=portfolio,xS = Sm, xr=r, xT=T)
        delta = (xp - xm)/(0.02*S[i])        
       
        d.append(delta)
    
    
    
    d = np.matrix(d).reshape(len(portfolio),1)   
    b = -np.matmul(L.transpose(),d)
    den = np.matmul(b.transpose(),b)[0,0]
    mu =  (x-a)*b/den
    return mu
    

In [68]:
sigma = np.array([x.sigma for x in portfolio])
rho = np.matrix([[1.0,0.8,0.8],
                  [0.8,1.0,0.8],
                  [0.8,0.8,1.0]
                  ])
d = np.diag(sigma)
cov = np.matmul(np.matmul(d,rho),d)

loss99 = 1000.0
mu = get_approx_mu(sigma,rho,loss99)
print(f"mu={mu}")
g = get_simulation_cube(nAssets=3, nSims=10, mu=mu.transpose(),Cov=cov)
print(g)



[[0.0225 0.042  0.054 ]
 [0.042  0.1225 0.126 ]
 [0.054  0.126  0.2025]]
[[0.0225 0.042  0.054 ]
 [0.042  0.1225 0.126 ]
 [0.054  0.126  0.2025]]
mu=[[-1.89208262]
 [-0.79187374]
 [-0.58425116]]
ff=-1.8725331624882156
ff=-3.176838343898443
ff=-1.4003766638742694
ff=-4.443715108946925
ff=-1.7400440150975696
ff=3.554461254443049
ff=-5.491299201305732
ff=-4.372187491501087
ff=-1.1364985328924844
ff=-4.027988062573934
(array([1.53733736e-01, 4.17173427e-02, 2.46504097e-01, 1.17521966e-02,
       1.75512675e-01, 3.49689755e+01, 4.12248476e-03, 1.26235963e-02,
       3.20940820e-01, 1.78101268e-02]), matrix([[-0.1651576 , -0.42022051, -0.07339325, -0.49409005, -0.19578383,
          0.04475595, -0.43239284, -0.36875705, -0.26886469, -0.35933374],
        [-0.80022699, -0.93630688, -0.52637799, -0.98914051, -0.72483572,
          0.24273718, -1.13790998, -0.9222729 , -0.49733562, -1.09553131],
        [-0.76377899, -0.92048237, -0.92875491, -1.32066441, -0.75369119,
          0.24616778, -1.6

In [60]:
rn = norm.rvs(size=(3,2)) 
print(rn)
print(mu.transpose())
print(rn+mu.transpose())

[[ 0.29612028  0.26105527]
 [ 0.00511346 -0.23458713]
 [-1.41537074 -0.42064532]]
[[10]
 [50]
 [95]]
[[10.29612028 10.26105527]
 [50.00511346 49.76541287]
 [93.58462926 94.57935468]]


In [ ]:
rn